# Two-Particle Self-Consistent approach (TPSC) tutorial

This tutorial is done in five steps:

1. You will first construct the non-interacting Green function of a square lattice
with nearest-neighbour hopping and convince yourself that the corresponding Fermi surface has perfect nesting [01]

2. You will compute the Lindhard function for the non-interacting susceptibility [03]

3. You will then compute the RPA approximation to check the divergence at ($\pi,\pi)$ [05]

4. You will compute renormalized spin and charge vertices in TPSC and see that RPA does not satisfy the Pauli principle [07]
   
5. You will finally compute the spin susceptibility to show that it does not diverge at finite temperature. [09]


In [ ]:
# Imports 
%matplotlib inline
from triqs.lattice import BravaisLattice, BrillouinZone
from triqs.gf import Gf, MeshProduct, MeshBrZone, MeshImFreq, Idx
from triqs.plot.mpl_interface import plt
import numpy as np
import warnings 
warnings.filterwarnings("ignore") #ignore some matplotlib warnings
from math import cos, pi
plt.rcParams["figure.figsize"] = (10,9) # set default size for all figures

## Lattice non-interacting Green function

In this notebook, we will consider a square lattice with nearest-neighbour hopping $t$.
The dispersion relation for this model is simply:

\begin{equation}
  \epsilon(\mathbf{k})=-2t(\cos{k_x}+\cos{k_y}),
\end{equation}

where $\mathbf{k}$ is a vector in the Brillouin zone (in units where the lattice spacing is unity $a=1$). The non-interacting Green's function for this lattice is given by

\begin{equation}
  G_0(\mathbf{k}, i\omega_n) = \frac{1}{i\omega_n - \epsilon(\mathbf{k})}
\end{equation}

### <i class="fa fa-gear fa-x" style="color: #186391"></i> Exercise 1:

Following the notebook introducing multivariable Green functions, create an
object `g0` that contains the non-interacting Green function $G_0(\mathbf{k},i\omega_n)$
of this lattice. You can use the parameters given below.

In [ ]:
# Regroup some parameters of the computation used later

beta = 1/0.4 # Inverse temperature
t = 1.0      # Hopping   
n_k = 128    # Number of points in the Brillouin Zone mesh (for each dimension)
n_iw = 128   # Number of Matsubara frequencies
mu = 0       # Chemical potential

# Write your code here, it should eventually define a variable g0

# ....

### <i class="fa fa-gear fa-x" style="color: #186391"></i> Exercise 2:

Save the Green function `g0` into an archive `tpsc.h5`, as `g0_kw`. This will be useful because
we will use `g0` later in other notebooks.

## For nearest-neighbor model, the Fermi surface is nested

Your goal here is to display the Fermi surface and see that it has perfect
nesting, meaning that large parts of the Fermi surface are mapped on to each other by a single momentum transfer $\mathbf{k}_{nesting}$, where $\mathbf{k}_{nesting}$ is called the "nesting vector".

### <i class="fa fa-gear fa-x" style="color: #186391"></i> Exercise 3:

Make a color plot of $-\frac{1}{\pi} Im G_0(\mathbf{k}, i\omega_0)$ over the
Brillouin zone.
For simplicity, we will neglect the fact that the first Matsubara frequency
$i\omega_0$ is not exactly $0$ at finite temperature and approximate the spectral function
at $k$ and $\omega=0$ by this quantity.

Hint: Here is an example of a code that makes a color plot of the
function $k_x^2 + k_y^2$. You can use it as a model to write your code.

```python
func = lambda kx, ky: kx**2+ky**2

kgrid1d = np.linspace(-np.pi, np.pi, 100, endpoint=True)
kx, ky = np.meshgrid(kgrid1d, kgrid1d) 
plt.pcolor(kx, ky, np.vectorize(func)(kx,ky))
```

You should see from the plot that the Fermi surface is **nested**:

  * What do we mean by that?
  * What is the nesting vector?

### <i class="fa fa-gear fa-x" style="color: #186391"></i> Exercise 4:

Plot the momentum distribution $n_\mathbf{k}$ along a diagonal of the Brillouin zone.

Hint 1: in order to obtain the density for a given value of $\mathbf{k}$ you
may want to:

  - Do a partial evaluation (see introduction to multivariable Green functions)
    of `g0` at the vector $\mathbf{k}$
  - Use the `density()` method studied in previous tutorials on this partially evaluated Green function
  
Hint 2: Here is a code to plot a function `func` of $k$ along the diagonal of the Brillouin
zone:

```python
func = lambda k: k**2
kgrid1d = np.linspace(-np.pi, np.pi, 100, endpoint=True)
plt.plot(kgrid1d, np.vectorize(func)(kgrid1d))
```

NB: In the above piece of code, `kgrid1d` is an array, so you cannot apply `func` directly to it because it is a function acting on scalars. In order for `func` to act on every element of an array, you should first "vectorize" it through `np.vectorize`.

Can you see a signature of the Fermi surface on this plot?